In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

creditos = pd.read_parquet('creditos.parquet', engine='pyarrow')
demografico = pd.read_parquet('demografico.parquet', engine='pyarrow')
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Challenge") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.sql("SELECT cr.customerid, cr.nombre_transaccion, cr.monto_transaccion, de.maritalstatus, de.gender, de.edad FROM parquet.`creditos.parquet` as cr INNER JOIN parquet.`demografico.parquet` as de ON cr.customerid == de.customerid ORDER BY cr.monto_transaccion DESC")
df.show()

+----------+------------------+-----------------+-------------+------+----+
|customerid|nombre_transaccion|monto_transaccion|maritalstatus|gender|edad|
+----------+------------------+-----------------+-------------+------+----+
|     40214|      SUPER AVANCE|       5370569.00|       CASADO|     M|74.0|
|     40214|      SUPER AVANCE|       5370569.00|       CASADO|     M|74.0|
|      5120|      SUPER AVANCE|       5370569.00|       CASADO|     M|72.0|
|     23189|       AVANCE CAJA|       5370569.00|       CASADO|     F|71.0|
|     87958|      SUPER AVANCE|       5370569.00|      SOLTERO|     F|72.0|
|     11271|      SUPER AVANCE|       5005371.00|       CASADO|     F|75.0|
|     81587|      SUPER AVANCE|       5000000.00|       CASADO|     F|75.0|
|     81587|      SUPER AVANCE|       5000000.00|       CASADO|     F|75.0|
|     44542|      SUPER AVANCE|       5000000.00|       CASADO|     M|70.0|
|     41946|       AVANCE CAJA|       5000000.00|       CASADO|     M|75.0|
|     47981|

In [2]:
pdf = df.toPandas()
pdf.shape

(3420, 6)

In [3]:
pdf.loc[pdf['nombre_transaccion'] == 'AVANCE CAJA', 'nombre_transaccion_map'] = 0
pdf.loc[pdf['nombre_transaccion'] == 'SUPER AVANCE', 'nombre_transaccion_map'] = 1

pdf.loc[pdf['monto_transaccion'] <= 1000000, 'monto_transaccion_map'] = 0
pdf.loc[(pdf['monto_transaccion'] > 1000000) & (pdf['monto_transaccion'] <= 2000000), 'monto_transaccion_map'] = 1
pdf.loc[(pdf['monto_transaccion'] > 2000000) & (pdf['monto_transaccion'] <= 3000000), 'monto_transaccion_map'] = 2
pdf.loc[(pdf['monto_transaccion'] > 3000000) & (pdf['monto_transaccion'] <= 4000000), 'monto_transaccion_map'] = 3
pdf.loc[(pdf['monto_transaccion'] > 4000000) & (pdf['monto_transaccion'] <= 5000000), 'monto_transaccion_map'] = 4
pdf.loc[pdf['monto_transaccion'] > 5000000, 'monto_transaccion_map'] = 5

pdf.loc[pdf['edad'] <= 40, 'edad_map'] = 0
pdf.loc[(pdf['edad'] > 41) & (pdf['edad'] <= 60), 'edad_map'] = 1
pdf.loc[(pdf['edad'] > 61) & (pdf['edad'] <= 80), 'edad_map'] = 2
pdf.loc[(pdf['edad'] > 81) & (pdf['edad'] <= 100), 'edad_map'] = 3
pdf.loc[pdf['edad'] > 100, 'edad_map'] = 4

pdf.loc[pdf['maritalstatus'] == 'SOLTERO', 'maritalstatus_map'] = 0
pdf.loc[pdf['maritalstatus'] == 'CASADO', 'maritalstatus_map'] = 1
pdf.loc[pdf['maritalstatus'] == 'VIUDO', 'maritalstatus_map'] = 2
pdf.loc[pdf['maritalstatus'] == 'NO INF', 'maritalstatus_map'] = 3

pdf.loc[pdf['gender'] == 'F', 'gender_map'] = 0
pdf.loc[pdf['gender'] == 'M', 'gender_map'] = 1

In [4]:
pdf.dtypes

customerid                  int32
nombre_transaccion         object
monto_transaccion          object
maritalstatus              object
gender                     object
edad                      float64
nombre_transaccion_map    float64
monto_transaccion_map     float64
edad_map                  float64
maritalstatus_map         float64
gender_map                float64
dtype: object

In [5]:
drop_elements = ['customerid', 'monto_transaccion', 'nombre_transaccion',
       'maritalstatus', 'gender', 'edad']
pdf_map = pdf.drop(drop_elements, axis = 1)
pdf_map

,nombre_transaccion_map,monto_transaccion_map,edad_map,maritalstatus_map,gender_map
0,0.0,5.0,2.0,1.0,0.0
1,1.0,5.0,2.0,0.0,0.0
2,1.0,5.0,2.0,1.0,1.0
3,1.0,5.0,2.0,1.0,1.0
4,1.0,5.0,2.0,1.0,1.0
5,1.0,5.0,2.0,1.0,0.0
6,1.0,4.0,2.0,1.0,0.0
7,0.0,4.0,2.0,1.0,0.0
8,0.0,4.0,2.0,1.0,1.0
9,1.0,4.0,2.0,1.0,0.0


In [6]:
pdf_map.isnull().sum()

nombre_transaccion_map      0
monto_transaccion_map       0
edad_map                  172
maritalstatus_map           0
gender_map                  0
dtype: int64

In [7]:
pdf_map['nombre_transaccion_map'] = pdf_map.nombre_transaccion_map.astype(int)

In [8]:
pdf_map['edad_map'].fillna(pdf_map['edad_map'].median(), inplace=True)
pdf_map.isnull().sum()

nombre_transaccion_map    0
monto_transaccion_map     0
edad_map                  0
maritalstatus_map         0
gender_map                0
dtype: int64

In [9]:
pdf_map['monto_transaccion_map'] = pdf_map.monto_transaccion_map.astype(int)

In [10]:
pdf_map['edad_map'] = pdf_map.edad_map.astype(int)

In [11]:
pdf_map['maritalstatus_map'] = pdf_map.maritalstatus_map.astype(int)

In [12]:
pdf_map['gender_map'] = pdf_map.gender_map.astype(int)

In [13]:
pdf_map

,nombre_transaccion_map,monto_transaccion_map,edad_map,maritalstatus_map,gender_map
0,0,5,2,1,0
1,1,5,2,0,0
2,1,5,2,1,1
3,1,5,2,1,1
4,1,5,2,1,1
5,1,5,2,1,0
6,1,4,2,1,0
7,0,4,2,1,0
8,0,4,2,1,1
9,1,4,2,1,0


In [14]:
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

kf = KFold(n_splits=10)
accuracies = list()
max_attributes = len(list(pdf_map))
depth_range = range(1, max_attributes + 1)
 
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='entropy',min_samples_split=20,min_samples_leaf=5,max_depth = depth,class_weight={1:18})
    for train_fold, valid_fold in kf.split(pdf_map):
        f_train = pdf_map.loc[train_fold] 
        f_valid = pdf_map.loc[valid_fold] 
        model = tree_model.fit(X = f_train.drop(['nombre_transaccion_map'], axis=1),y = f_train['nombre_transaccion_map']) 
        valid_acc = model.score(X = f_valid.drop(['nombre_transaccion_map'], axis=1),y = f_valid['nombre_transaccion_map'])
        fold_accuracy.append(valid_acc)
    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)

dfr = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
dfr = dfr[["Max Depth", "Average Accuracy"]]
print(dfr.to_string(index=False))

Max Depth  Average Accuracy
        1          0.946491
        2          0.946491
        3          0.946491
        4          0.946491
        5          0.946491


In [15]:
from subprocess import check_call
import pydot
y_train = pdf_map['nombre_transaccion_map']
x_train = pdf_map.drop(['nombre_transaccion_map'], axis=1).values 
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', min_samples_split=20, min_samples_leaf=5,max_depth = depth,class_weight={1:18})
decision_tree.fit(x_train, y_train)
with open(r"tree1.dot", 'w') as f:
     f = tree.export_graphviz(decision_tree,out_file=f,max_depth = 7,impurity = True,feature_names = list(pdf_map.drop(['nombre_transaccion_map'], axis=1)), class_names = ['AVANCE CAJA', 'SUPER AVANCE'], rounded = True,filled= True)
check_call(['dot','-Tpng',r'tree1.dot','-o',r'tree1.png'])
PImage("tree1.png")        

In [16]:
edad = 72
maritalstatus = 'SOLTERO'
gender = 'M'

if(edad <= 40):
    edad_map = 0
elif ((edad > 41) and (edad <= 60)):
    edad_map = 1
elif ((edad > 61) and (edad <= 80)):
    edad_map = 2 
elif ((edad > 81) and (edad <= 100)):
    edad_map = 3    
else:
    edad_map = 4  
    
if(maritalstatus == 'SOLTERO'):
    maritalstatus_map = 0
elif (maritalstatus == 'CASADO'):
    maritalstatus_map = 1
elif (maritalstatus == 'VIUDO'):
    maritalstatus_map = 2    
else:
    maritalstatus_map = 3   
    
if(gender == 'F'):
    gender_map = 0
else:
    gender_map = 1   
    
nt_map = pdf_map[(pdf_map['edad_map'] == edad_map) & (pdf_map['maritalstatus_map'] == maritalstatus_map) & (pdf_map['gender_map'] == gender_map)].groupby('nombre_transaccion_map').size()
mt_map = pdf_map[(pdf_map['edad_map'] == edad_map) & (pdf_map['maritalstatus_map'] == maritalstatus_map) & (pdf_map['gender_map'] == gender_map)].groupby('monto_transaccion_map').size()
nt_map = nt_map.index[len(nt_map.index)-1]
mt_map = mt_map.index[len(mt_map.index)-1]
    
if(nt_map == 0):
    tipo_de_credito = "AVANCE CAJA"
else:
    tipo_de_credito = "SUPER AVANCE"

if(mt_map == 0):
    monto = 1000000
elif(mt_map == 1):
    monto = 2000000
elif(mt_map == 2):
    monto = 3000000
elif(mt_map == 3):
    monto = 4000000
elif(mt_map == 4):
    monto = 5000000
elif(mt_map == 5):
    monto = 5370569
    
print("Tipo de crédito aprobado: " + str(tipo_de_credito) + ", Monto: " + str(monto))

x_test = pd.DataFrame(columns=('nombre_transaccion_map','monto_transaccion_map', 'edad_map', 'maritalstatus_map','gender_map'))
print(str(nt_map) + ", " + str(mt_map) + ", " + str(edad_map) + ", " + str(maritalstatus_map) + ", " + str(gender_map))
x_test.loc[0] = (nt_map,mt_map,edad_map,maritalstatus_map,gender_map)
y_pred = decision_tree.predict(x_test.drop(['nombre_transaccion_map'], axis = 1))
if(y_pred[0]==1):
    res = "Aprobado"
else:
    res = "Rechazado"
print("Decisión: " + res)
y_proba = decision_tree.predict_proba(x_test.drop(['nombre_transaccion_map'], axis = 1))
print("Con probabilidad: " + str(round(y_proba[0][y_pred][0]* 100, 2))+"%")

Tipo de crédito aprobado: SUPER AVANCE, Monto: 5000000
1, 4, 2, 0, 1
Decisión: Aprobado
Con probabilidad: 98.39%


In [17]:
edad = 55
maritalstatus = 'CASADO'
gender = 'F'

if(edad <= 40):
    edad_map = 0
elif ((edad > 41) and (edad <= 60)):
    edad_map = 1
elif ((edad > 61) and (edad <= 80)):
    edad_map = 2 
elif ((edad > 81) and (edad <= 100)):
    edad_map = 3    
else:
    edad_map = 4  
    
if(maritalstatus == 'SOLTERO'):
    maritalstatus_map = 0
elif (maritalstatus == 'CASADO'):
    maritalstatus_map = 1
elif (maritalstatus == 'VIUDO'):
    maritalstatus_map = 2    
else:
    maritalstatus_map = 3   
    
if(gender == 'F'):
    gender_map = 0
else:
    gender_map = 1   
    
nt_map = pdf_map[(pdf_map['edad_map'] == edad_map) & (pdf_map['maritalstatus_map'] == maritalstatus_map) & (pdf_map['gender_map'] == gender_map)].groupby('nombre_transaccion_map').size()
mt_map = pdf_map[(pdf_map['edad_map'] == edad_map) & (pdf_map['maritalstatus_map'] == maritalstatus_map) & (pdf_map['gender_map'] == gender_map)].groupby('monto_transaccion_map').size()
nt_map = nt_map.index[len(nt_map.index)-1]
mt_map = mt_map.index[len(mt_map.index)-1]
    
if(nt_map == 0):
    tipo_de_credito = "AVANCE CAJA"
else:
    tipo_de_credito = "SUPER AVANCE"

if(mt_map == 0):
    monto = 1000000
elif(mt_map == 1):
    monto = 2000000
elif(mt_map == 2):
    monto = 3000000
elif(mt_map == 3):
    monto = 4000000
elif(mt_map == 4):
    monto = 5000000
elif(mt_map == 5):
    monto = 5370569
    
print("Tipo de crédito aprobado: " + str(tipo_de_credito) + ", Monto: " + str(monto))

x_test = pd.DataFrame(columns=('nombre_transaccion_map','monto_transaccion_map', 'edad_map', 'maritalstatus_map','gender_map'))
print(str(nt_map) + ", " + str(mt_map) + ", " + str(edad_map) + ", " + str(maritalstatus_map) + ", " + str(gender_map))
x_test.loc[0] = (nt_map,mt_map,edad_map,maritalstatus_map,gender_map)
y_pred = decision_tree.predict(x_test.drop(['nombre_transaccion_map'], axis = 1))
if(y_pred[0]==1):
    res = "Aprobado"
else:
    res = "Rechazado"
print("Decisión: " + res)
y_proba = decision_tree.predict_proba(x_test.drop(['nombre_transaccion_map'], axis = 1))
print("Con probabilidad: " + str(round(y_proba[0][y_pred][0]* 100, 2))+"%")

Tipo de crédito aprobado: SUPER AVANCE, Monto: 2000000
1, 1, 1, 1, 0
Decisión: Aprobado
Con probabilidad: 95.02%
